In [1]:
from bs4 import BeautifulSoup
import requests
import json
import re
import string
from pymongo import MongoClient
from pymongo.server_api import ServerApi
from nltk.corpus import stopwords
import pyarabic.araby as araby
from pyarabic.number import ArNumbers
from tashaphyne.stemming import ArabicLightStemmer
import qalsadi.lemmatizer as lem
import pandas as pd 

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0'}
req = requests.get('https://www.hespress.com/?s=%D8%B3%D8%AA%D8%A7%D9%84%D9%8A%D9%86', headers=headers)
doc = BeautifulSoup(req.text, 'html.parser')

In [12]:
%pip install selenium


   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.5 MB 640.0 kB/s eta 0:00:15
   ---------------------------------------- 0.0/9.5 MB 640.0 kB/s eta 0:00:15
   ---------------------------------------- 0.1/9.5 MB 558.5 kB/s eta 0:00:17
   ---------------------------------------- 0.1/9.5 MB 581.0 kB/s eta 0:00:17
   ---------------------------------------- 0.1/9.5 MB 544.7 kB/s eta 0:00:18
   ---------------------------------------- 0.1/9.5 MB 544.7 kB/s eta 0:00:18
   ---------------------------------------- 0.1/9.5 MB 544.7 kB/s eta 0:00:18
   ---------------------------------------- 0.1/9.5 MB 544.7 kB/s eta 0:00:18
    --------------------------------------- 0.2/9.5 MB 503.2 kB/s eta 0:00:19
   - -------------------------------------- 0.3/9.5 MB 541.3 kB/s eta 0:00:18
   - -------------------------------------- 0.3/9.5 MB 570.1 kB/s eta 0:00:17
   - -------------------------------------- 0.3/9.5 MB 561.6 kB/s eta 0:0

scarp articles in relation with stalin 

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

# Initialize the Selenium WebDriver
driver = webdriver.Chrome()  # You may need to specify the path to the driver if it's not in your PATH
driver.get('https://www.hespress.com/?s=%D8%B3%D8%AA%D8%A7%D9%84%D9%8A%D9%86')

# Scroll to the bottom to load dynamic content
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Adjust delay time as needed
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Parse the fully loaded page
html = driver.page_source
doc = BeautifulSoup(html, 'html.parser')

# Your existing logic to extract articles
main_divs = doc.find_all(name='div', attrs={'class': 'search_results row'})
articals = []
for main_div in main_divs:
    posts = main_div.find_all(name='div', attrs={'class': 'card-img-top'})
    for article in posts:
        try:
            link = article.find(name='a').get('href')
            title = article.find(name='a').get('title')
            articals.append({title: link})
        except AttributeError:
            continue

# Check how many articles are successfully stored
print(f"Total number of articles stored: {len(articals)}")

# Close the WebDriver
driver.quit()


Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


Total number of articles stored: 195


In [14]:
articals


[{'في الحزب عفن وخونة..': 'https://www.hespress.com/%d8%b3%d9%8a%d8%af%d9%8a-%d9%81%d9%8a-%d8%a7%d9%84%d8%ad%d8%b2%d8%a8-%d8%b9%d9%81%d9%86-%d9%88%d8%ae%d9%88%d9%86%d8%a9-%d9%88%d8%ac%d8%a8-%d8%aa%d8%b7%d9%87%d9%8a%d8%b1%d9%87-1340166.html'},
 {'أردوغان: إسرائيل دولة إرهابية تنفذ الإبادة': 'https://www.hespress.com/%d8%a3%d8%b1%d8%af%d9%88%d8%ba%d8%a7%d9%86-%d8%a5%d8%b3%d8%b1%d8%a7%d8%a6%d9%8a%d9%84-%d8%af%d9%88%d9%84%d8%a9-%d8%a5%d8%b1%d9%87%d8%a7%d8%a8%d9%8a%d8%a9-%d9%88%d9%86%d8%aa%d9%86%d9%8a%d8%a7%d9%87-1327561.html'},
 {'مفهوم "القيادة الجماعية"': 'https://www.hespress.com/%d9%81%d9%8a-%d8%a7%d9%84%d8%ad%d8%a7%d8%ac%d8%a9-%d8%a5%d9%84%d9%89-%d8%aa%d8%ad%d8%af%d9%8a%d8%af-%d9%85%d9%81%d9%87%d9%88%d9%85-%d8%a7%d9%84%d9%82%d9%8a%d8%a7%d8%af%d8%a9-%d8%a7%d9%84%d8%ac%d9%85-1325169.html'},
 {'الانتصار بـــ" غسيل الفندق"': 'https://www.hespress.com/%d8%a7%d9%84%d8%a7%d9%86%d8%aa%d8%b5%d8%a7%d8%b1-%d8%a8%d9%80%d9%80%d9%80-%d8%ba%d8%b3%d9%8a%d9%84-%d8%a7%d9%84%d9%81%d9%86%d8%af%d9%82-1313

In [15]:
import json
import os
# Save articles and their links to JSON file
output_dir = 'data'
os.makedirs(output_dir, exist_ok=True)
with open(os.path.join(output_dir, 'hess_article_links.json'), 'w', encoding='utf-8') as f:
    data = []
    for article in articals:
        title = list(article.keys())[0]
        link = list(article.values())[0]
        data.append({'title': title, 'link': link})
    json.dump(data, f, ensure_ascii=False, indent=2)


In [16]:
# Read the saved JSON file to fetch article content
with open(os.path.join(output_dir, 'hess_article_links.json'), 'r', encoding='utf-8') as f:
    article_content = []
    data = json.load(f)
    for article in data:
        try:
            article_result = requests.get(article['link'], headers=headers)
            article_result.raise_for_status()
            article_doc = BeautifulSoup(article_result.text, 'html.parser')
            content = article_doc.find_all(name='div', attrs={'class': 'article-content'})
            if content:
                paragraphs = content[0].find_all(name='p')
                article_paragraphs = [p.text for p in paragraphs]
                article_content.append({
                    'title': article['title'],
                    'content': ' '.join(article_paragraphs)
                })
        except (requests.RequestException, AttributeError) as e:
            print(f"Error fetching content for {article['title']}: {e}")


In [17]:
# Save the article content to another JSON file
with open(os.path.join(output_dir, 'hess_article_content.json'), 'w', encoding='utf-8') as f:
    json.dump(article_content, f, ensure_ascii=False, indent=2)

print("Articles and their content have been successfully saved.")

Articles and their content have been successfully saved.


articlas m3ndomch 3ala9a m3a stalin 

In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

# Initialize the Selenium WebDriver
driver = webdriver.Chrome()  # You may need to specify the path to the driver if it's not in your PATH
driver.get('https://www.hespress.com/?s=%D8%B9%D9%8A%D8%AF+%D8%A7%D9%84%D8%A7%D8%B6%D8%AD%D9%89')

# Scroll to the bottom to load dynamic content
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Adjust delay time as needed
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Parse the fully loaded page
html = driver.page_source
doc2 = BeautifulSoup(html, 'html.parser')

# Your existing logic to extract articles
main_divs2 = doc2.find_all(name='div', attrs={'class': 'search_results row'})
non_articals = []
for main_div in main_divs2:
    posts2 = main_div.find_all(name='div', attrs={'class': 'card-img-top'})
    for article in posts2:
        try:
            link = article.find(name='a').get('href')
            title = article.find(name='a').get('title')
            non_articals.append({title: link})
        except AttributeError:
            continue

# Check how many articles are successfully stored
print(f"Total number of articles stored: {len(non_articals)}")

# Close the WebDriver
driver.quit()


Total number of articles stored: 30


In [27]:
import json
import os

with open(os.path.join(output_dir, 'hess_non_article_links.json'), 'w', encoding='utf-8') as f:
    data = []
    for article in non_articals:
        title = list(article.keys())[0]
        link = list(article.values())[0]
        data.append({'title': title, 'link': link})
    json.dump(data, f, ensure_ascii=False, indent=2)

In [28]:
# Read the saved JSON file to fetch non_article content
with open(os.path.join(output_dir, 'hess_non_article_links.json'), 'r', encoding='utf-8') as f:
    non_article_content = []
    data = json.load(f)
    for article in data:
        try:
            article_result = requests.get(article['link'], headers=headers)
            article_result.raise_for_status()
            article_doc = BeautifulSoup(article_result.text, 'html.parser')
            content = article_doc.find_all(name='div', attrs={'class': 'article-content'})
            if content:
                paragraphs = content[0].find_all(name='p')
                article_paragraphs = [p.text for p in paragraphs]
                non_article_content.append({
                    'title': article['title'],
                    'content': ' '.join(article_paragraphs)
                })
        except (requests.RequestException, AttributeError) as e:
            print(f"Error fetching content for {article['title']}: {e}")


In [30]:
# Save the non_article content to another JSON file
with open(os.path.join(output_dir, 'hess_non_article_content.json'), 'w', encoding='utf-8') as f:
    json.dump(non_article_content, f, ensure_ascii=False, indent=2)

print("Articles and their content have been successfully saved.")

Articles and their content have been successfully saved.


In [58]:
import json
import csv
import numpy as np
# Function to calculate relevance score
def calculate_score(text):
    # Criteria for scoring: length of text and keyword presence
    length_score = min(len(text) / 10000, 1) * 3  # Length score (up to 3 points)
    keywords = ['الاشتراكية ','سوفياتي' ,'ستالين', 'روسيا']
    keyword_score = sum(text.lower().count(keyword) for keyword in keywords) * 1.5  # Keyword score (up to 5 points)
    total_score = min(length_score + keyword_score, 10)  # Ensure the score doesn't exceed 10
    return np.floor(total_score)

# Path to your existing JSON file
json_file_path = f'C:\Users\lenovo\Desktop\master\S2\nlp\atelier4\data\hess_article_content.json'

# Read the JSON file to fetch article content
with open(json_file_path, 'r', encoding='utf-8') as f:
    article_content = json.load(f)

# List to store the article text and their respective scores
articles_with_scores = []

# Calculate the score for each article and store it in a list
for article in article_content:
    text = article['content']
    score = calculate_score(text)
    articles_with_scores.append({'text': text, 'score': score})

# Path to save the CSV file
csv_file_path = r'C:\Users\lenovo\Desktop\master\S2\nlp\atelier4\data\hess_article_content.csv'

# Save the article content and scores to a CSV file
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['text', 'score']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for article in articles_with_scores:
        writer.writerow(article)

print("Articles and their content with scores have been successfully saved to CSV.")

Articles and their content with scores have been successfully saved to CSV.


In [59]:
df=pd.read_csv("./data/hess_article_content.csv")

In [60]:
df

,text,score
0,ذات يوم قال ستالين، وهو بالمناسبة أبشع أمين عا...,10.0
1,قال الرئيس التركي رجب طيب أردوغان إن بنيامين ن...,2.0
2,دخل حزب الأصالة والمعاصرة في الآونة الأخيرة، م...,4.0
3,استطاع الفلسطينيون أن يحققوا انتصارا كبيرا على...,5.0
4,لا يزال جثمان لينين المحنط مسجى في ضريحه في ال...,10.0
...,...,...
220,شرعية(بمعنى الحفاظ على استقرار الوطن المغربي) ...,3.0
221,أعلن مؤخرا الحبيب المالكي (الصورة) عضو المكتب ...,10.0
222,كان الكثير من المغاربة يُقدرون أن الحسن الثان...,9.0
223,أربعون سنة مرت على ثورة الشباب في اروبة في ماي...,10.0
